---
# Part 1: Data Preprocessing
---

## Step 1: Merge the Data
---

- Import necessary packages.

In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
import re
import os, fnmatch
import zipfile
from datetime import datetime, timedelta
from IPython.display import display

- Create Helpers.

In [2]:
def walklevel(some_dir, level=0):
    some_dir = some_dir.rstrip(os.path.sep)
    assert os.path.isdir(some_dir)
    num_sep = some_dir.count(os.path.sep)
    for root, dirs, files in os.walk(some_dir):
        yield root, dirs, files
        num_sep_this = root.count(os.path.sep)
        if num_sep + level <= num_sep_this:
            del dirs[:]
            
def swap_day_month(s):
    day, month, yr = s.split('.')
    return month +'/' + day +'/' + yr

def swap_day_month2(s):
    yr, month, day = s.split('-')
    return month +'/' + day +'/' + yr

def create_hr_seq():
    now = datetime(2000, 1, 1, 0, 0, 0)
    last = datetime(2000, 1, 2, 0, 0, 0)
    delta = timedelta(seconds=1)
    times = []
    while now < last:
        times.append(repr(now.strftime('%H:%M:%S')))
        now += delta
    return times

def create_dtype_dict():
    dic = {}
    dic['Unnamed: 0'] = str
    for i in create_hr_seq():
        dic[i] = float
    return dic

- Iterate through each house:
  - Create a master DataFrame;
  - Process *occupancy* data:
    -
  - Process *plugs* data:

In [3]:
### master DataFrames of each house
house_df_dict = {}

CUR_PATH = './DataSets'
change_path = lambda DIR : CUR_PATH + '/' + DIR

for root, dirs, files in walklevel(CUR_PATH): 
    ### 6 Houses
    for house in dirs:
        house_number = int(house.split(' ')[1])
        print('...' + house + '...')
        CUR_PATH = change_path(house)
        # read .txt file
        for file in os.listdir(CUR_PATH):  
            if fnmatch.fnmatch(file, '*.txt'):
                with open(change_path(file)) as f:
                    F = f.read()
                    num_appliance_str = re.findall(r'[0-9][0-9]:\s[^\(]*[a-z]', F)
                    dates = re.findall(r'[0-9][0-9]\.[0-9][0-9]\.[0-9][0-9]', F)
            
        # calculate data range from .txt file
        start_date, end_date = swap_day_month(dates[2]), swap_day_month(dates[3]) #Plugs
        # create the master df for the house
        master_df = pd.DataFrame(pd.date_range(start=start_date, end=end_date, freq='S', closed='left'),
                                 columns=['date']) #Plugs>Occupancy
        ### Process Occupancy Data
        if house_number != 6: # not available
            print('Processing Occupancy Data...')
            ROOT1 = CUR_PATH
            CUR_PATH = change_path('Occupany') if house_number != 1 else change_path('Occupancy')
            # unzip 01_occupancy_csv.zip (NOT IMPLEMENTED YET)
            CUR_PATH = change_path('{:02}_occupancy_csv'.format(house_number))
            ROOT2 = CUR_PATH
            # read summer/winter.csv
            csv_tuple = []
            for file in os.listdir(CUR_PATH):  
                if fnmatch.fnmatch(file, '*.csv'):
                    csv_df = pd.read_csv(change_path(file), dtype=create_dtype_dict())
                    # transpose csv_df
                    csv_df = csv_df.set_index('Unnamed: 0')
                    csv_df.index.name = ''
                    csv_df = csv_df.T 
                    csv_tuple.append(csv_df)
            assert csv_tuple[0].shape[0] == csv_tuple[1].shape[0]
            combined_csv = pd.concat(csv_tuple, axis=1)       
            # convert to 2 cols
            combined_csv.index.name = 'date'
            concat_lst = []
            for col in combined_csv.columns:
                df = combined_csv[col].to_frame()
                df.rename(columns={col:'foo'}, inplace=True)
                df.reset_index(level=0, inplace=True)
                df['date'] = pd.date_range(start=col, periods=60*60*24, freq='S')
                concat_lst.append(df)
            # add to master_df
            final_df = pd.concat(concat_lst).rename(columns={'foo':'occupancy'})
            master_df = master_df.merge(final_df, on='date', how='left')
            CUR_PATH = ROOT1
        
        
        
        ### Process Plug Data
        print('Processing Plugs Data...')
        CUR_PATH = change_path('Plugs')
#         # unzip (NEED TO BE MODIFIED: doesn't store zipped folder)
#         for file in os.listdir(CUR_PATH)  :  
#             if fnmatch.fnmatch(file, '*.zip'):
#                 import zipfile
#                 zip = zipfile.ZipFile(change_path(file), 'r')
#                 zip.extractall()
#                 print('unzipped ' + file) ### cannot unzip and save the subdirectory
        CUR_PATH = change_path('{:02}'.format(house_number))
        ROOT = CUR_PATH
        # get the names of the each electric appliance
        num_appliance_dict = dict(item.split(": ") for item in num_appliance_str)
        # loop through each appliance
        for n in num_appliance_dict.keys():
            appliance_name = num_appliance_dict[n]
            CUR_PATH = change_path(n)
            concat_lst = []
            # loop through each date of this appliance
            done = [] # prevent duplicates from House 02/09
            for file in os.listdir(CUR_PATH):  
                if fnmatch.fnmatch(file, '*.csv'):
                    date = swap_day_month2(os.path.splitext(file)[0][:10])
                    if date in done:
                        continue 
                    else:
                        done.append(date)
                    csv_df = pd.read_csv(change_path(file), names=[appliance_name])
                    cur_df = pd.DataFrame(pd.date_range(start=date, periods=60*60*24, freq='S'), columns=['date'])
                    cur_df[appliance_name] = csv_df.iloc[:,0]
                    concat_lst.append(cur_df)
            df = pd.concat(concat_lst, ignore_index=True)
            master_df = master_df.merge(df, on='date', how='left')
            CUR_PATH = ROOT
                        
        ### Process Metor Data (not now)
        house_df_dict[house_number] = master_df
        CUR_PATH = './DataSets'
        print('Completed {}!'.format(house))
print('Successfully merged all the data!!!')

###House 3...
Processing Occupancy Data...
Processing Plugs Data...
Completed House 3!
###House 4...
Processing Occupancy Data...
Processing Plugs Data...
Completed House 4!
###House 5...
Processing Occupancy Data...
Processing Plugs Data...
Completed House 5!
###House 2...
Processing Occupancy Data...
Processing Plugs Data...
Completed House 2!
###House 1...
Processing Occupancy Data...
Processing Plugs Data...
Completed House 1!
###House 6...
Processing Plugs Data...
Completed House 6!
Successfully merged all the data!!!


- Check the created master DataFrames.

In [4]:
for key in sorted(house_df_dict.keys()):
    df = house_df_dict[key]
    print('House {:02d} has {} millions of rows with the preview:'.format(key, round(df.shape[0]/1000000, 2)))
    display(df.head())
    print('Its features/columns are:')
    print(*df.columns, sep=", ") 
    print('')
    print('The general data statistics are:')
    display(df.describe())
    print('The Data Frame information (null, data type, etc.) are:')
    display(df.info())

House 01 has 2.42 millions of rows with the preview:


,date,occupancy,Fridge,Dryer,Coffee machine,Kettle,Washing machine,Freezer
0,2012-06-01 00:00:00,NaN,49.2516,830.508,NaN,0.0,4.39739,2.23178
1,2012-06-01 00:00:01,NaN,49.2516,834.774,NaN,0.0,4.39739,2.23178
2,2012-06-01 00:00:02,NaN,49.2516,834.774,NaN,0.0,4.39739,2.23178
3,2012-06-01 00:00:03,NaN,51.3899,832.641,NaN,0.0,4.39739,2.23178
4,2012-06-01 00:00:04,NaN,49.2516,832.641,NaN,0.0,6.53380,2.23178


Its features/columns are:
date, occupancy, Fridge, Dryer, Coffee machine, Kettle, Washing machine, Freezer

The general data statistics are:


,occupancy,Fridge,Dryer,Coffee machine,Kettle,Washing machine,Freezer
count,0.0,2.419200e+06,2.419200e+06,0.0,2.160000e+06,2.419200e+06,2.419200e+06
mean,NaN,2.185438e+01,2.369411e+01,NaN,3.593142e+00,2.317556e+01,1.859330e+01
std,NaN,2.532282e+01,1.333412e+02,NaN,8.151185e+01,1.818361e+02,1.561907e+01
min,NaN,-1.000000e+00,-1.000000e+00,NaN,-1.000000e+00,-1.000000e+00,-1.000000e+00
25%,NaN,2.205780e+00,0.000000e+00,NaN,0.000000e+00,0.000000e+00,2.231780e+00
50%,NaN,4.344320e+00,0.000000e+00,NaN,0.000000e+00,0.000000e+00,3.027450e+01
75%,NaN,4.711320e+01,0.000000e+00,NaN,0.000000e+00,4.397390e+00,3.243160e+01
max,NaN,9.849370e+02,9.648820e+02,NaN,1.891870e+03,2.387580e+03,8.204040e+01


The Data Frame information (null, data type, etc.) are:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2419200 entries, 0 to 2419199
Data columns (total 8 columns):
date               datetime64[ns]
occupancy          float64
Fridge             float64
Dryer              float64
Coffee machine     float64
Kettle             float64
Washing machine    float64
Freezer            float64
dtypes: datetime64[ns](1), float64(7)
memory usage: 166.1 MB


...

,date,Lamp,Laptop,Router,Coffee machine,Entertainment,Fridge,Kettle
0,2012-06-27 00:00:00,0.0,4.35384,19.3387,0.0,15.0043,2.19884,0.0
1,2012-06-27 00:00:01,0.0,4.35384,19.3387,0.0,15.0043,2.19884,0.0
2,2012-06-27 00:00:02,0.0,6.47995,19.3387,0.0,15.0043,0.00000,0.0
3,2012-06-27 00:00:03,0.0,6.47995,19.3387,0.0,15.0043,0.00000,0.0
4,2012-06-27 00:00:04,0.0,4.35384,19.3387,0.0,15.0043,0.00000,0.0


Its features/columns are:
date, Lamp, Laptop, Router, Coffee machine, Entertainment, Fridge, Kettle

The general data statistics are:


,Lamp,Laptop,Router,Coffee machine,Entertainment,Fridge,Kettle
count,1.425600e+07,1.589760e+07,7.603200e+06,1.537920e+07,1.555200e+07,1.537920e+07,1.270080e+07
mean,-8.376633e-02,6.586103e+00,1.917793e+01,3.643533e+00,2.341736e+01,1.011450e+01,2.354713e+00
std,2.567623e+00,6.320948e+00,3.825831e+00,4.375191e+01,2.186640e+01,3.181019e+01,7.143318e+01
min,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00
25%,-1.000000e+00,4.353840e+00,1.933870e+01,0.000000e+00,1.500430e+01,0.000000e+00,0.000000e+00
50%,0.000000e+00,6.479950e+00,1.933870e+01,0.000000e+00,1.500430e+01,0.000000e+00,0.000000e+00
75%,0.000000e+00,6.479950e+00,1.933870e+01,0.000000e+00,2.139790e+01,2.198840e+00,0.000000e+00
max,4.729930e+01,9.364450e+01,2.786790e+01,1.285580e+03,1.535160e+02,1.128580e+03,2.122290e+03


The Data Frame information (null, data type, etc.) are:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 18835200 entries, 0 to 18835199
Data columns (total 8 columns):
date              datetime64[ns]
Lamp              float64
Laptop            float64
Router            float64
Coffee machine    float64
Entertainment     float64
Fridge            float64
Kettle            float64
dtypes: datetime64[ns](1), float64(7)
memory usage: 1.3 GB


None

## Step 2: Clean the Data
---

### (a) Deal with `nan` and `-1` values

### (b) ...

---
# Part 2: EDA
---

---
# Part 3: Time Series Spike
---

---
# Part 4: ML
---

---
# Part 5: Linear Programming
---